1. Download the ([zip](https://github.com/mulongocheloti/Python-SQLite3-Connection/raw/main/data.rar)) file, unzip and copy .csv file into your working directory. Read it in.<br>
   Interested in only two fields; **'Countries'** and **'2017 vs 1990: change(%)'**<br>
   Do any relevant transformations.<br>
2. Using SQL,
*   Create a database instance.
*   Create a table named **Emmission** with two fields  named **country** and **%_change(1990-2017)**
*   From the selected columns, Insert relevant records into the two fields of the table Emmission and save the changes.
*   Sort the table; with the country with the highest %_change(1990-2017) on top.
<br>
3. Display only the top 10 countries.<br>
4. Plot a graph.

In [1]:
import sqlite3
import pandas as pd
import plotly.express as px

df = pd.read_csv("/content/SQLLite_pandas_data.csv")
df.head()

,Unnamed: 0,Countries,2017 vs 1990: change(%)
0,0,World,63.5%
1,1,World - International Aviation,109.8%
2,2,World - International Shipping,82.2%
3,3,Afghanistan,348.6%
4,4,Albania,-23.7%


In [2]:
df['2017 vs 1990: change(%)'] = df['2017 vs 1990: change(%)'].str.rstrip('%').astype('float')

df['2017 vs 1990: change(%)'] = df['2017 vs 1990: change(%)'].round(3)

df.head()

,Unnamed: 0,Countries,2017 vs 1990: change(%)
0,0,World,63.5
1,1,World - International Aviation,109.8
2,2,World - International Shipping,82.2
3,3,Afghanistan,348.6
4,4,Albania,-23.7


In [3]:
column_1 = df['Countries'].tolist()
column_2 = df['2017 vs 1990: change(%)'].tolist()

# create the database instance
conn = sqlite3.connect('Country_Change.db')

# Create a cursor
cur = conn.cursor()

# Create the table named Emission
cur.execute("DROP TABLE IF EXISTS Emission;")
cur.execute("CREATE TABLE Emission (Country text, `%_change(1990-2017)` float);")

# Insert the records
cur.executemany("INSERT into Emission values (?, ?)", zip(column_1, column_2))

# Commit all the transactions
conn.commit()

In [4]:
# sort
sorted_df = pd.read_sql_query("SELECT * FROM Emission ORDER BY `%_change(1990-2017)` DESC", conn)

# filter only top 10
top_10_countries = sorted_df.head(10)

# plot
fig = px.bar(top_10_countries, x='Country', y='%_change(1990-2017)')
fig.show()